In [1]:
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 KB 840.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 KB 3.2 MB/s eta 0:00:00a 0:00:01


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
import torch
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import optuna
from datetime import datetime

2024-01-10 00:25:39.104334: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 00:25:39.778355: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-10 00:25:39.778426: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-10 00:25:41.691202: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [4]:
# Load the dataset
df = pd.read_csv("./content/train.csv")  # Replace with the actual path to your dataset

# Encode labels using scikit-learn's LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['store_type'])

# Split the dataset into training, validation, and testing sets
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [8]:
# Define a custom dataset class
class ItemDataset(Dataset):
    def __init__(self, items, labels, tokenizer, max_len):
        self.items = items
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        item = str(self.items[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            item,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'item': item,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(int(label), dtype=torch.long)  # Convert label to integer
        }

# Set device (GPU if available, else CPU)
device = 'cpu' #torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initilize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['store_type'].unique()))
model.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [9]:
# Training Function
MAX_LEN = 32

def train(epochs, lr, train_loader, valid_loader):
  """Train Model"""

  model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['store_type'].unique()))

  # Loss function and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.AdamW(model.parameters(), lr=lr)

  # Training loop with validation
  for epoch in range(epochs):
      model.train()
      total_loss = 0

      for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs} - Training', leave=False):
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['label'].to(device)

          optimizer.zero_grad()

          outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
          loss = outputs.loss
          loss.backward()
          optimizer.step()

          total_loss += loss.item()

      # Calculate average training loss
      average_train_loss = total_loss / len(train_loader)

      # Validation
      model.eval()
      all_preds = []
      all_labels = []

      with torch.no_grad():
          for batch in tqdm(valid_loader, desc=f'Epoch {epoch + 1}/{epochs} - Validation', leave=False):
              input_ids = batch['input_ids'].to(device)
              attention_mask = batch['attention_mask'].to(device)
              labels = batch['label'].to(device)

              outputs = model(input_ids, attention_mask=attention_mask)
              logits = outputs.logits
              preds = torch.argmax(logits, dim=1).cpu().numpy()

              all_preds.extend(preds)
              all_labels.extend(labels.cpu().numpy())

      # Calculate validation accuracy
      accuracy = accuracy_score(all_labels, all_preds)
      print(f'Epoch {epoch + 1}/{epochs} - Training Loss: {average_train_loss:.4f} - Validation Accuracy: {accuracy:.4f}')
  return accuracy


In [ ]:
# Hyper Parameter Search

def objective(trial):
  epochs = trial.suggest_int('epochs', 3, 10)
  lr = trial.suggest_float('learning_rate', 1e-7, 1e-3, log=True)
  batch_size = trial.suggest_int('batch_size', 5, 10)

  # Dataloaders
  train_dataset = ItemDataset(train_df['item'].values, train_df['label'].values, tokenizer, MAX_LEN)
  valid_dataset = ItemDataset(valid_df['item'].values, valid_df['label'].values, tokenizer, MAX_LEN)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

  # Run Training
  accuracy = train(epochs, lr, train_loader, valid_loader)

  return accuracy

# Create the Optuna study
study = optuna.create_study(direction='maximize')

# Run the optimization
study.optimize(objective, n_trials=10)

# Print the results
print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial
params={}
print('Value: ', trial.value)
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')
    params.update({key: value})

pd.DataFrame([params]).to_csv(f"best_params_{datetime.now()}.csv")


In [21]:
# Best parameter training
#print(params)
params = {'epochs': 3, 'learning_rate': 8.161247251036099e-05, 'batch_size': 8}
train_dataset = ItemDataset(train_df['item'].values, train_df['label'].values, tokenizer, MAX_LEN)
valid_dataset = ItemDataset(valid_df['item'].values, valid_df['label'].values, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=params["batch_size"], shuffle=False)

train(params["epochs"], params["learning_rate"], train_loader, valid_loader)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1/3 - Training Loss: 2.8064 - Validation Accuracy: 0.2456


Epoch 2/3 - Training Loss: 2.1182 - Validation Accuracy: 0.4737


Epoch 3/3 - Training Loss: 1.3803 - Validation Accuracy: 0.6316


0.631578947368421

In [13]:
# Testing loop
model.eval()
all_preds = []
all_labels = []
test_dataset = ItemDataset(test_df['item'].values, test_df['label'].values, tokenizer, MAX_LEN)
test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)

with torch.no_grad():
    for batch in tqdm(train_loader, desc=f'Testing', leave=False):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

# Evaluation
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy:.2f}')

print(classification_report(all_labels, all_preds))

Accuracy: 0.00
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       3.0
           2       0.00      0.00      0.00      21.0
           3       0.00      0.00      0.00      16.0
           4       0.00      0.00      0.00       7.0
           5       0.00      0.00      0.00       2.0
           6       0.00      0.00      0.00       4.0
           7       0.00      0.00      0.00       1.0
           8       0.00      0.00      0.00      28.0
           9       0.00      0.00      0.00      20.0
          10       0.00      0.00      0.00      20.0
          11       0.00      0.00      0.00      25.0
          12       0.00      0.00      0.00      51.0
          13       0.00      0.00      0.00       4.0
          14       0.00      0.00      0.00      14.0
          15       0.00      0.00      0.00       3.0
          16       0.00      0.00      0.00       0.0
          17       0.00      0.00      0.00      20.0
          18

/home/ben/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ben/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ben/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ben/.local/lib/python3.10/site-packages/sklearn/

In [16]:
def predict_store_type(model, tokenizer, label_encoder, item, max_len=16):
    model.eval()

    with torch.no_grad():
        encoding = tokenizer.encode_plus(
            item,
            add_special_tokens=True,
            max_length=max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        output = model(input_ids, attention_mask=attention_mask)
        logits = output.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    # Decode the predicted label using the label encoder
    predicted_store_type = label_encoder.inverse_transform([predicted_label])[0]

    return predicted_store_type

# Assuming 'model', 'tokenizer', and 'label_encoder' are already defined and loaded
item_to_predict = "apples"
predicted_store = predict_store_type(model, tokenizer, label_encoder, item_to_predict)
print(f"The predicted store type for '{item_to_predict}' is: {predicted_store}")


The predicted store type for 'apples' is: hair_care


In [22]:
inputs = tokenizer("apples", return_tensors="pt")
label_mapping = {i: label for i, label in enumerate(label_encoder.classes_)}
        # Forward pass through the model
outputs = model(**inputs)
print(label_mapping)
# Get the predicted label
predicted_label = torch.argmax(outputs.logits, dim=1).item()
predicted_tag = label_mapping[int(predicted_label)]
predicted_tag

{0: 'atm', 1: 'bakery', 2: 'beauty_salon', 3: 'bicycle_store', 4: 'book_store', 5: 'cafe', 6: 'car_repair', 7: 'car_wash', 8: 'clothing_store', 9: 'convenience_store', 10: 'department_store', 11: 'drugstore', 12: 'electronics_store', 13: 'florist', 14: 'furniture_store', 15: 'gas_station', 16: 'hair_care', 17: 'hardware_store', 18: 'home_goods_store', 19: 'jewelry_store', 20: 'liquor_store', 21: 'pet_store', 22: 'pharmacy', 23: 'shoe_store', 24: 'supermarket'}


'hair_care'

In [ ]:
# Save model
model.save_pretrained("/content/model")
tokenizer.save_pretrained("/content/tokenizer")

label_mapping = {i: label for i, label in enumerate(label_encoder.classes_)}
import json
json_file_path = "/content/label_mapping.json"

# Write the dictionary to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(label_mapping, json_file, indent=4)

In [ ]:
# Compress model files for export
import tarfile

def compress_files(file_paths, zip_file_path):
    with tarfile.open(zip_file_path, 'w:gz') as tar:
        for file_path in file_paths:
            # Add each file to the tarball
            tar.add(file_path, arcname=file_path.lstrip('/'))

# Example usage
file_paths = ["/content/model/config.json", "/content/tokenizer", "/content/label_mapping.json"]
zip_file_path = "/content/compressed_files.tar.gz"

compress_files(file_paths, zip_file_path)

